In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('./dataFolders/Output/MothChart-LightLevel-FlowerShape.xlsx', sheet_name='Sheet1_copy_forIncidenceOfFligh')

In [7]:
df.head()

,Date,Animal Name,Light Level,Eclosed Date,Date Caught,"Age (at time of expt, in days)",Gender,Body Length (cm),Proboscis Length (cm),Total trials,Successful trials,Comments,Person,Unnamed: 13
0,2018_07_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,L3_c-2_m1,2.85,2018_7_28-29,2018_07_31,2 or 3,Male,4.7,7.8,21.0,19.0,"Cover for light fell off during experiment, re...",Mahad,NaN
2,NaN,L0.3_c-3_m1,0.30,2018_7_28-29,2018_07_31,2 or 3,Female,4.8,9.0,26.0,26.0,NaN,Mahad,NaN
3,NaN,L0.03_c-1_m1,0.04,2018_7_28-29,2018_07_31,2 or 3,Male,4.7,8.0,0.0,0.0,NaN,Mahad,NaN
4,2018_08_02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# remove all data after 2018_10_23 # date after which we started collecting "data" for final paper

def f(x):
    m = x['Date'].eq('2018_10_23')
    b = m.idxmax()
    if m.any():
        x = x.loc[:b]
    else:
        x
    return x

In [26]:
new_df = f(df)
new_df.iloc[-1,:]

Date                              2018_10_23
Animal Name                              NaN
Light Level                              NaN
Eclosed Date                             NaN
Date Caught                              NaN
Age (at time of expt, in days)           NaN
Gender                                   NaN
Body Length (cm)                         NaN
Proboscis Length (cm)                    NaN
Total trials                             NaN
Successful trials                        NaN
Comments                                 NaN
Person                                   NaN
Unnamed: 13                              NaN
Name: 108, dtype: object

In [27]:
subset = new_df.dropna(subset = ['Light Level'], axis = 0)
subset.dropna(subset = ['Total trials'], axis = 0, inplace = True)
dataset = subset.loc[:, ['Animal Name', 'Total trials', 'Successful trials']].copy()

C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
tt = dataset['Animal Name'].str.split('_', n = 2, expand = True)
dataset['lightLevel'] = tt[0]
dataset['lightLevel'] = dataset['lightLevel'].str[1:]
dataset['flowerType'] = tt[1]

dataset[['Total trials', 'Successful trials']] =  dataset[['Total trials', 'Successful trials']].astype('int')
dataset.head()

,Animal Name,Total trials,Successful trials,lightLevel,flowerType
1,L3_c-2_m1,21,19,3,c-2
2,L0.3_c-3_m1,26,26,0.3,c-3
3,L0.03_c-1_m1,0,0,0.03,c-1
5,L0.3_c-10_m1,11,2,0.3,c-10
6,L0.3_c-3_m2,20,20,0.3,c-3


In [29]:
dataset.loc[dataset['Total trials'] == 0, 'incidence'] = False
dataset.loc[dataset['Total trials'] > 0, 'incidence'] = True

In [30]:
l = []
total = []
flew = []
fraction = []

for ll, bla in dataset.groupby('lightLevel'):
    total_moths = len(bla)
    moth_that_flew = bla.incidence.sum()
        
    l.append(ll)
    total.append(total_moths)
    flew.append(moth_that_flew)
    
summary = pd.DataFrame({'lightLevel': l,
                       'totalMoths': total,
                       'mothThatFlew': flew})
summary['fraction_flew']=summary.mothThatFlew/summary.totalMoths

In [31]:
summary['lightLevel'] = summary.lightLevel.astype('float')
soreted = summary.sort_values(by = ['lightLevel'])

In [32]:
soreted

,lightLevel,totalMoths,mothThatFlew,fraction_flew
0,0.03,1,0,0.000000
1,0.06,2,0,0.000000
2,0.10,8,4,0.500000
3,0.30,33,24,0.727273
4,3.00,27,20,0.740741
6,50.00,8,5,0.625000
7,65.00,1,1,1.000000
5,300.00,5,0,0.000000


In [33]:
trans = soreted.drop(axis = 0, index = 7)
trans

,lightLevel,totalMoths,mothThatFlew,fraction_flew
0,0.03,1,0,0.000000
1,0.06,2,0,0.000000
2,0.10,8,4,0.500000
3,0.30,33,24,0.727273
4,3.00,27,20,0.740741
6,50.00,8,5,0.625000
5,300.00,5,0,0.000000


In [34]:
import matplotlib.pyplot as plt
plt.show()

f = plt.figure(figsize = (3.5, 3.5))
plt.plot(np.log10(trans.lightLevel), trans.fraction_flew, 'o-')

plt.xticks(np.log10(trans.lightLevel).values, ['0.03', '0.06', '0.10', '0.30', '3.00', '50.00', '300.00'])
plt.xlabel('log10 (light level) in lux ')
plt.ylabel('Flight incidence')

f.savefig('./dataFolders/Output/Step6_v4/Figure/FlightIncidence_beforeExperimentStarted.pdf')

In [111]:
np.log10(trans.lightLevel).values

array([-1.52287875, -1.22184875, -1.        , -0.52287875,  0.47712125,
        1.69897   ,  2.47712125])